In [881]:
def updateTable(data,gameNum,table):
    row = {}
    overview = data['itemList']['items'][gameNum]
    #Take care of broad things first
    compID = overview['competitionId']
    otherID = overview['id']
    try:
        live = overview['LIVE']
    except:
        live = False
#       return table
#   if not live:
#       return table
    awayTeamFirst = overview['awayTeamFirst']
    denySameGame = overview['denySameGame']

    homeTeam = overview['competitors'][0]['abbreviation']
    awayTeam = overview['competitors'][1]['abbreviation']

    firstCompFloor = overview['competitors'][0]['type']

    if firstCompFloor=="AWAY":
        temp = homeTeam
        homeTeam = awayTeam
        awayTeam =temp
    score = np.nan
    row.update({'compID':compID,'otherID':otherID,
                'homeTeam':homeTeam,'awayTeam':awayTeam,
                'currentTime':time,'score':score})

    #Now specific data
    spreadDat = overview['displayGroups'][0]['itemList'][0]['outcomes']
    lineDat  = overview['displayGroups'][0]['itemList'][1]['outcomes']
    ouDat  = overview['displayGroups'][0]['itemList'][2]['outcomes']

    #Spread Data
    spreadStatus = spreadDat[0]['status']

    spreadHome = spreadDat[1]['price']['handicap']
    spreadAway = spreadDat[0]['price']['handicap']

    spreadLineHome = spreadDat[1]['price']['decimal']
    spreadLineAway = spreadDat[0]['price']['decimal']

    row.update({'spreadHome':spreadHome,'spreadLineHome':spreadLineHome,
                'spreadAway':spreadAway,'spreadLineAway':spreadLineAway,
                'spreadStatus':spreadStatus})
    #Money Line

    lineStatus = lineDat[0]['status']
    lineHome = lineDat[1]['price']['decimal']
    lineAway = lineDat[0]['price']['decimal']
    row.update({'lineStatus':lineStatus, 'moneylineHome':lineHome,'moneylineAway':lineAway})

    #OU dat

    ouStatus = lineDat[0]['status']

    ouLine = ouDat[0]['price']['handicap']

    ouLineOver = ouDat[0]['price']['decimal']
    ouLineUnder = ouDat[1]['price']['decimal']

    row.update({'ouLine':ouLine, 
                'ouLineOver':ouLineOver,'ouLineUnder':ouLineUnder,
                'ouStatus':ouStatus})

    row.update({'live':live,'denySameGame':denySameGame,'awayTeamFirst':awayTeamFirst})
    rowDf = pd.DataFrame(row,index=[1])
    
    return(pd.concat([table,rowDf],ignore_index=True))

In [858]:
user_agent = "\"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9\""
# header variable
headers = { 'User-Agent' : user_agent }
page = requests.get('https://sports.bovada.lv/basketball/nba-playoffs',headers=headers)
tree = html.fromstring(page.content)
raw = tree.xpath('//script[@type="text/javascript"]/text()')[-1]

data = json.loads(raw.split("=")[1])['items'][0]

time = datetime.datetime.now()



In [882]:
fullDf = pd.DataFrame(columns=rowDf.columns)
fullDf
nGames = len(data['itemList']['items'])
for i in range(0,nGames):
    fullDf = updateTable(data,i,fullDf)
fullDf

,awayTeam,awayTeamFirst,compID,currentTime,denySameGame,homeTeam,lineStatus,live,moneylineAway,moneylineHome,...,ouLine,ouLineOver,ouLineUnder,ouStatus,score,spreadAway,spreadHome,spreadLineAway,spreadLineHome,spreadStatus
0,UTA,True,2824553,2018-05-01 22:55:04.032319,NO,HOU,OPEN,False,6.250,1.1176,...,207.0,1.870,1.952381,OPEN,NaN,11.0,-11.0,1.909091,1.909091,OPEN
1,CLE,True,2824553,2018-05-01 22:55:04.032319,NO,TOR,OPEN,False,3.200,1.384615,...,212.0,1.870,1.952381,OPEN,NaN,6.5,-6.5,1.909091,1.909091,OPEN
2,PHI,True,2824553,2018-05-01 22:55:04.032319,NO,BOS,OPEN,False,1.606061,2.450,...,207.0,1.909091,1.909091,OPEN,NaN,-3.5,3.5,1.952381,1.870,OPEN


So next time we get this we want to work on a way to do this continuously
pull data, wait a little bit, pull more data.

Other things we want to do
figure out how to get score and time

potential way
also pull from nba website or somewhere else a current score and time
alternatively a score and time kind of data map after the game

on route 1.
we should look into some sort of feed data, how do we work with that streaming data
find some stream and anytime a score happens pull bovada for an update



In [97]:
%%time
useragent = "\"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9\""
gameurl = "https://sports.bovada.lv/basketball/nba-playoffs/cleveland-cavaliers-toronto-raptors-201805012000"
currentdata = !wget -q -O - --user-agent={useragent} {gameurl}


CPU times: user 612 µs, sys: 6.17 ms, total: 6.79 ms
Wall time: 965 ms


In [576]:
%%time
user_agent = "\"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9\""
# header variable
headers = { 'User-Agent' : user_agent }

page = requests.get('https://sports.bovada.lv/basketball/nba-playoffs',\
                   headers=headers)
tree = html.fromstring(page.content)


CPU times: user 22.4 ms, sys: 0 ns, total: 22.4 ms
Wall time: 920 ms


In [468]:
a = json.loads(l.replace("var swc_game_view = ",""))

In [469]:
import pandas as pd
b = tree.xpath('//script[@type="text/javascript"]/text()')[-1]
fullerset =json.loads(b.split("=")[1])['items'][0]

In [90]:
page = requests.get('http://econpy.pythonanywhere.com/ex/001.html')
tree2= html.fromstring(page.content)
tree2.xpath('//div[@title="buyer-name"]/text()')

['Carson Busses',
 'Earl E. Byrd',
 'Patty Cakes',
 'Derri Anne Connecticut',
 'Moe Dess',
 'Leda Doggslife',
 'Dan Druff',
 'Al Fresco',
 'Ido Hoe',
 'Howie Kisses',
 'Len Lease',
 'Phil Meup',
 'Ira Pent',
 'Ben D. Rules',
 'Ave Sectomy',
 'Gary Shattire',
 'Bobbi Soks',
 'Sheila Takya',
 'Rose Tattoo',
 'Moe Tell']

In [130]:
currentdata[-4]

'                <iframe uia-src="//casino.bovada.lv/static/site10/files/websites_cross_domain/storage.html" class="cross-domain-iframe" style="display:none;"></iframe> <iframe uia-src="//poker.bovada.lv/static/site10/files/websites_cross_domain/storage.html" class="cross-domain-iframe" style="display:none;"></iframe> <iframe uia-src="//horses.bovada.lv/static/site10/files/websites_cross_domain/storage.html" class="cross-domain-iframe" style="display:none;"></iframe> <iframe uia-src="//www.bovada.lv/static/site10/files/websites_cross_domain/storage.html" class="cross-domain-iframe" style="display:none;"></iframe>                <script type="text/javascript" src="https://cdn13.staticfilesdelivery.com/static/site10/js/js_WIRpHrlzOD_gs7TjmoJVJL9_G2xtjgS-gRClpVybFI8.js"></script>'

In [129]:
currentdata

['<!DOCTYPE html>',
 '<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="en" version="XHTML+RDFa 1.0" dir="ltr"',
 '  xmlns:og="http://ogp.me/ns#"',
 '  xmlns:article="http://ogp.me/ns/article#"',
 '  xmlns:book="http://ogp.me/ns/book#"',
 '  xmlns:profile="http://ogp.me/ns/profile#"',
 '  xmlns:video="http://ogp.me/ns/video#"',
 '  xmlns:product="http://ogp.me/ns/product#" ng-class="uaClasses">',
 '<!--[if lt IE 7]>      <html class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->',
 '<!--[if IE 7]>         <html class="no-js lt-ie9 lt-ie8"> <![endif]-->',
 '<!--[if IE 8]>         <html class="no-js lt-ie9"> <![endif]-->',
 '<!--[if gt IE 8]>      <html class="no-js"> <![endif]-->',
 '<head profile="http://www.w3.org/1999/xhtml/vocab">',
 '    <!--[if IE]><![endif]-->',
 '<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />',
 '<meta http-equiv="x-dns-prefetch-control" content="on" />',
 '<link rel="dns-prefetch" href="//cdn13.staticfilesdelivery.com" />',
 '<!--[if IE 9]

In [141]:
%%time
useragent = "\"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9\""
gameurl = "https://sports.bovada.lv/basketball/nba-playoffs/cleveland-cavaliers-toronto-raptors-201805012000"
currentdata = !wget -q -O - --user-agent={useragent} {gameurl} | grep "<script type=\"text/javascript\""

CPU times: user 2.68 ms, sys: 988 µs, total: 3.66 ms
Wall time: 1.01 s


In [315]:
a['items'][0]

{'LIVE': True,
 'atmosphereLink': '/api/atmosphere/eventNotification/events/A/3179318',
 'awayTeamFirst': True,
 'baseLink': '/basketball/nba-playoffs/cleveland-cavaliers-toronto-raptors-201805012000',
 'competitionId': '2824553',
 'competitors': [{'abbreviation': 'TOR',
   'description': 'Toronto Raptors',
   'id': '3179318-275',
   'rotationNumber': '558',
   'shortName': 'Raptors',
   'type': 'HOME'},
  {'abbreviation': 'CLE',
   'description': 'Cleveland Cavaliers',
   'id': '3179318-1028',
   'rotationNumber': '557',
   'shortName': 'Cavaliers',
   'type': 'AWAY'}],
 'denySameGame': 'NO',
 'description': 'Cleveland Cavaliers @ Toronto Raptors',
 'displayGroups': [{'baseLink': '/basketball/nba-playoffs/game-lines-market-group',
   'defaultType': True,
   'description': 'Game Lines',
   'id': '97',
   'itemList': [{'belongsToDefault': True,
     'columns': 'H2Columns',
     'description': 'Point Spread',
     'displayGroups': 'A-97,97',
     'id': '46275073',
     'isInRunning': Tru

In [313]:
a['items'][0].keys()

dict_keys(['description', 'id', 'link', 'baseLink', 'atmosphereLink', 'type', 'LIVE', 'awayTeamFirst', 'denySameGame', 'feedCode', 'competitionId', 'competitors', 'notes', 'numMarkets', 'sport', 'startTime', 'status', 'displayGroups'])

In [240]:
a['items'][0]['displayGroups'][0].keys()

dict_keys(['description', 'id', 'link', 'baseLink', 'defaultType', 'itemList'])

In [245]:
a['items'][0]['displayGroups'][0]['itemList'][0].keys()

dict_keys(['description', 'id', 'type', 'columns', 'isInRunning', 'mainMarketType', 'mainPeriod', 'marketTypeGroup', 'sequence', 'sportCode', 'periodType', 'status', 'displayGroups', 'notes', 'belongsToDefault', 'outcomes'])

In [258]:
a['items'][0]['displayGroups'][0]['itemList'][1]['outcomes']

[{'competitorId': '3179318-1028',
  'description': 'Cleveland Cavaliers',
  'id': '206929418',
  'price': {'american': '+375',
   'decimal': '4.750',
   'fractional': '15/4',
   'id': '998809794',
   'outcomeId': '206929418'},
  'status': 'OPEN',
  'type': 'A'},
 {'competitorId': '3179318-275',
  'description': 'Toronto Raptors',
  'id': '206929410',
  'price': {'american': '-650',
   'decimal': '1.153846',
   'fractional': '2/13',
   'id': '998809795',
   'outcomeId': '206929410'},
  'status': 'OPEN',
  'type': 'H'}]

In [ ]:
oneList = {}

In [362]:
fullerset['itemList']['items'][0].keys()

dict_keys(['description', 'id', 'link', 'baseLink', 'atmosphereLink', 'type', 'LIVE', 'awayTeamFirst', 'denySameGame', 'feedCode', 'competitionId', 'competitors', 'notes', 'numMarkets', 'sport', 'startTime', 'status', 'displayGroups'])

In [371]:
fullerset['itemList']['items'][0]['LIVE']

True

In [373]:
fullerset['itemList']['items'][0]['awayTeamFirst']

True

In [372]:
fullerset['itemList']['items'][0]['id']

'3177926'

In [382]:
fullerset['itemList']['items'][0]['competitionId']

'2824553'

In [389]:
fullerset['itemList']['items'][0]['competitors'][0]['abbreviation']

'GSW'

In [391]:
fullerset['itemList']['items'][0]['denySameGame']

'NO'

In [611]:
fullerset['itemList']['items'][0]['displayGroups'][0].keys()

dict_keys(['description', 'id', 'link', 'baseLink', 'defaultType', 'itemList'])

In [614]:
fullerset['itemList']['items'][0]['displayGroups'][0]['itemList'][0]

{'belongsToDefault': True,
 'columns': 'H2Columns',
 'description': 'Point Spread',
 'displayGroups': 'A-97,97',
 'id': '46280334',
 'isInRunning': True,
 'mainMarketType': 'SPREAD',
 'mainPeriod': True,
 'marketTypeGroup': 'SPREAD',
 'notes': '',
 'outcomes': [{'competitorId': '3177926-11757494',
   'description': 'New Orleans Pelicans',
   'id': '206973905',
   'price': {'american': '-115',
    'decimal': '1.870',
    'fractional': '20/23',
    'handicap': '8.5',
    'id': '999001655',
    'outcomeId': '206973905'},
   'status': 'OPEN',
   'type': 'A'},
  {'competitorId': '3177926-11757531',
   'description': 'Golden State Warriors',
   'id': '206973904',
   'price': {'american': '-115',
    'decimal': '1.870',
    'fractional': '20/23',
    'handicap': '-8.5',
    'id': '999001654',
    'outcomeId': '206973904'},
   'status': 'OPEN',
   'type': 'H'}],
 'periodType': 'Live Match',
 'sequence': '9',
 'sportCode': 'BASK',
 'status': 'OPEN',
 'type': 'SPR'}

In [616]:
fullerset['itemList']['items'][0]['displayGroups'][0]['itemList'][0].keys()

dict_keys(['description', 'id', 'type', 'columns', 'isInRunning', 'mainMarketType', 'mainPeriod', 'marketTypeGroup', 'sequence', 'sportCode', 'periodType', 'status', 'displayGroups', 'notes', 'belongsToDefault', 'outcomes'])

In [618]:
fullerset['itemList']['items'][0]['displayGroups'][0]['itemList'][0]['outcomes'][0][]

{'competitorId': '3177926-11757494',
 'description': 'New Orleans Pelicans',
 'id': '206973905',
 'price': {'american': '-115',
  'decimal': '1.870',
  'fractional': '20/23',
  'handicap': '8.5',
  'id': '999001655',
  'outcomeId': '206973905'},
 'status': 'OPEN',
 'type': 'A'}

In [628]:
fullerset['itemList']['items'][0]['displayGroups'][0]['itemList'][1]['outcomes'][0]['price']

{'american': '+425',
 'decimal': '5.250',
 'fractional': '17/4',
 'id': '999001659',
 'outcomeId': '206973909'}

In [795]:
fullDf = pd.DataFrame(columns=rowDf.columns)

In [838]:
fullDf = updateTable(data,2,fullDf)
fullDf

,awayTeam,awayTeamFirst,compID,currentTime,denySameGame,homeTeam,lineStatus,live,moneylineAway,moneylineHome,...,ouLine,ouLineOver,ouLineUnder,ouStatus,score,spreadAway,spreadHome,spreadLineAway,spreadLineHome,spreadStatus
0,UTA,True,2824553,2018-05-01 22:49:37.122019,NO,HOU,OPEN,False,6.250,1.1176,...,207.0,1.870,1.952381,OPEN,NaN,11.0,-11.0,1.909091,1.909091,OPEN
1,UTA,True,2824553,2018-05-01 22:49:37.122019,NO,HOU,OPEN,False,6.250,1.1176,...,207.0,1.870,1.952381,OPEN,NaN,11.0,-11.0,1.909091,1.909091,OPEN
2,UTA,True,2824553,2018-05-01 22:49:37.122019,NO,HOU,OPEN,False,6.250,1.1176,...,207.0,1.870,1.952381,OPEN,NaN,11.0,-11.0,1.909091,1.909091,OPEN
3,CLE,True,2824553,2018-05-01 22:49:37.122019,NO,TOR,OPEN,False,3.200,1.384615,...,212.0,1.870,1.952381,OPEN,NaN,6.5,-6.5,1.909091,1.909091,OPEN
4,PHI,True,2824553,2018-05-01 22:49:37.122019,NO,BOS,OPEN,False,1.606061,2.450,...,207.0,1.909091,1.909091,OPEN,NaN,-3.5,3.5,1.952381,1.870,OPEN


In [770]:
print(fullDf.columns)
print(fullDf.values)

Index(['awayTeam', 'awayTeamFirst', 'compID', 'currentTime', 'denySameGame',
       'homeTeam', 'lineStatus', 'live', 'moneylineAway', 'moneylineHome',
       'otherID', 'ouLine', 'ouLineOver', 'ouLineUnder', 'ouStatus', 'score',
       'spreadAway', 'spreadHome', 'spreadLineAway', 'spreadLineHome',
       'spreadStatus'],
      dtype='object')
[['UTA' True '2824553' Timestamp('2018-05-01 22:17:54.452385') 'NO' 'HOU'
  'OPEN' False '6.250' '1.1176' '3177929' '205.0' '205.0' '205.0' 'OPEN'
  nan '11.0' '-11.0' '1.909091' '1.909091' 'OPEN']]


In [774]:
ouDat

[{'description': 'Over',
  'id': '205470658',
  'lineQualityColour': 'ff0000',
  'lineQualityDescription': 'Restricted',
  'price': {'american': '-115',
   'decimal': '1.870',
   'fractional': '20/23',
   'handicap': '205.0',
   'id': '998981881',
   'outcomeId': '205470658'},
  'status': 'OPEN',
  'type': 'O'},
 {'description': 'Under',
  'id': '205470659',
  'lineQualityColour': 'ff0000',
  'lineQualityDescription': 'Restricted',
  'price': {'american': '-105',
   'decimal': '1.952381',
   'fractional': '20/21',
   'handicap': '205.0',
   'id': '998981882',
   'outcomeId': '205470659'},
  'status': 'OPEN',
  'type': 'U'}]